In [65]:
from pynq import Overlay, MMIO
import time

conv_design = Overlay("./HW10-2.bit")



/usr/local/lib/python3.6/dist-packages/pynq/overlay.py:299: UserWarning: Users will not get PARAMETERS / REGISTERS information through TCL files. HWH file is recommended.
  warnings.warn(message, UserWarning)


In [66]:
cdma_addr = conv_design.ip_dict['axi_cdma_0']['phys_addr']
busy_addr = conv_design.ip_dict['axi_gpio_0']['phys_addr']
ready_addr = conv_design.ip_dict['axi_gpio_1']['phys_addr']

print("cdma_addr: 0x"+format(cdma_addr,'02x'))
print("busy_addr: 0x"+format(busy_addr,'02x'))
print("ready_addr: 0x"+format(ready_addr,'02x'))




cdma_addr: 0x7e200000
busy_addr: 0x41200000
ready_addr: 0x41210000


In [67]:
mem_addr = 0x30000000
bram0_addr = 0xC0000000

#4096 * 4
in_bytes = 16384
in_mem = MMIO(mem_addr, in_bytes)
cdma = MMIO(cdma_addr, out_bytes)
busy = MMIO(busy_addr, 8)
ready = MMIO(ready_addr, 8)


input_file = open('input.hex', 'r')
in_lines = input_file.readlines()
raddr = 0
counter = 0
for line in in_lines: 
    if((counter == 0)or(counter > 4096)):
        continue
    else:
        hex_string = line[0:8]
        to_int = int(hex_string, 16) & 0xFFFFFFFF  # Mask to 32 bits
        # Write to memory
        in_mem.write(raddr, to_int)
        # print(f"Writing {hex_string} to addr {raddr}")
        raddr += 4
    counter += 1
input_file.close()


cdma.write(0x00, 0x04)
cdma.write(0x18, mem_addr)
cdma.write(0x20, bram0_addr)
cdma.write(0x28, in_bytes)

ready.write(0x4, 0x0)  
ready.write(0x0, 0xF)  # GPIO output 1
#2048 * 4
out_bytes = 8192
out_mem = MMIO(mem_addr, out_bytes)

# CDMA move data from BRAM1 to system memory 
cdma.write(0x00, 0x04)
cdma.write(0x18, bram0_addr+57344)
cdma.write(0x20, mem_addr)
# Write out_bytes to BTT (offset: 0x28) to set the number of bytes to transfer
cdma.write(0x28, out_bytes)



In [68]:

# Read output from output.hex
golden_file = open('golden.hex', 'r')
g_lines = golden_file.readlines()
err = 0
addr = 0
for line in g_lines:
    expect = hex(int(line[0:8], 16))
    # Read from memory
    output = hex(out_mem.read(addr))
    if output != expect:
        # Remove '0x' from hex string and ensure both are properly formatted
        output_formatted = output[2:].upper().zfill(8)
        expect_formatted = expect[2:].upper().zfill(8)
        print(f"[Error] Output is {output_formatted} expected {expect_formatted} at addr {addr}")
        err += 1
    addr += 4

if err == 0:
    print("Congratulations! ALL pass")
else:
    print("Error! Data mismatch")

[Error] Output is 00000000 expected 00004906 at addr 4
[Error] Output is 00000000 expected 00005080 at addr 12
[Error] Output is 00000000 expected 00005926 at addr 20
[Error] Output is 00000000 expected 00006094 at addr 28
[Error] Output is 00000000 expected 000063C3 at addr 36
[Error] Output is 00000000 expected 0000669B at addr 44
[Error] Output is 00000000 expected 0000688D at addr 52
[Error] Output is 00000000 expected 00006ADA at addr 60
[Error] Output is 00000000 expected 00006C4A at addr 68
[Error] Output is 00000000 expected 00006E89 at addr 76
[Error] Output is 00000000 expected 000070D3 at addr 84
[Error] Output is 00000000 expected 00007233 at addr 92
[Error] Output is 00000000 expected 000073A3 at addr 100
[Error] Output is 00000000 expected 00007490 at addr 108
[Error] Output is 00000000 expected 0000758F at addr 116
[Error] Output is 00000000 expected 000075C1 at addr 124
[Error] Output is 00000000 expected 0000759F at addr 132
[Error] Output is 00000000 expected 0000759E

[Error] Output is 00000000 expected 000018ED at addr 3492
[Error] Output is 00000000 expected 000018C6 at addr 3496
[Error] Output is 00000000 expected 00000D79 at addr 3504
[Error] Output is 00000000 expected 00002947 at addr 3516
[Error] Output is 00000000 expected 00003D99 at addr 3524
[Error] Output is 00000000 expected 00003DC2 at addr 3532
[Error] Output is 00000000 expected 00003CA2 at addr 3540
[Error] Output is 00000000 expected 00003A0C at addr 3548
[Error] Output is 00000000 expected 00003779 at addr 3556
[Error] Output is 00000000 expected 00003562 at addr 3564
[Error] Output is 00000000 expected 000032D1 at addr 3572
[Error] Output is 00000000 expected 00005367 at addr 3576
[Error] Output is 00000000 expected 00003011 at addr 3580
[Error] Output is 00000000 expected 00002802 at addr 3588
[Error] Output is 00000000 expected 00002B23 at addr 3596
[Error] Output is 00000000 expected 00003166 at addr 3604
[Error] Output is 00000000 expected 0000365B at addr 3612
[Error] Output

[Error] Output is 00000000 expected 0000096C at addr 6800
[Error] Output is 00000000 expected 00000E2D at addr 6808
[Error] Output is 00000000 expected 0000095E at addr 6816
[Error] Output is 00000000 expected 000018DB at addr 6824
[Error] Output is 00000000 expected 00001F9D at addr 6832
[Error] Output is 00000000 expected 00001C16 at addr 6840
[Error] Output is 00000000 expected 00002535 at addr 6860
[Error] Output is 00000000 expected 00002527 at addr 6868
[Error] Output is 00000000 expected 000022EE at addr 6876
[Error] Output is 00000000 expected 00001F4D at addr 6884
[Error] Output is 00000000 expected 00001CCF at addr 6892
[Error] Output is 00000000 expected 00001B1E at addr 6900
[Error] Output is 00000000 expected 00004FE5 at addr 6904
[Error] Output is 00000000 expected 000018F2 at addr 6908
[Error] Output is 00000000 expected 00000D9B at addr 6916
[Error] Output is 00000000 expected 000010F7 at addr 6924
[Error] Output is 00000000 expected 000017FD at addr 6932
[Error] Output